# Replicación carteras de Carhart
## "On Persistence in Mutual Fund Performance" - MARK M. CARHART

In [2]:
# Importamos paquetes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series
import statsmodels.api as sm
from scipy import stats

In [14]:
# Importamos data
date = pd.read_csv('date.csv', header = None)
rf = pd.read_csv('rf.csv', header = None)
rmrf = pd.read_csv('rmrf.csv', header = None)
x = pd.read_csv('x.csv')

hml = pd.read_csv('hml.csv', header = None)
smb = pd.read_csv('smb.csv', header = None)
umd = pd.read_csv('umb.csv', header = None)
market = rmrf[0]

In [4]:
#Obtenemos el exceso de retorno restando los retornos brutos (x) menos el retorno libre de riesgo (rf)
ret_excess = np.subtract(x, rf)

# check
#ret_excess.iloc[11].dropna()

# Ordenamos en un DataFrame la información
df = pd.DataFrame(ret_excess)

# Se descartan todos los fondos con Nan antes de los primeros 12 meses
data1 = df.iloc[:12,:].dropna(1)

# Rentabilidad de los ultimos 12 meses para cada uno de los fondos
averageReturn = np.mean(data1)

# Se muestra el encabezado de la lista con sus respectivos retornos promedio
averageReturn.head()

Fund 25   -2.306725
Fund 30   -0.786175
Fund 51   -0.260583
Fund 52   -5.190625
Fund 76   -1.574675
dtype: float64

In [5]:
# Definimos funciones que nos ayudaran a realizar el estudio

# Funcion -boots- a partir de una base de datos "df" se obtiene los retornos de los fondos entre
# los periodos de partida "start" y fin "end" descartando todos los fondos con NaN entre esos periodos.
def boots(df, start, end):
    z = 0
    data = pd.DataFrame()
    ventana = []
    data = df.iloc[start:end,:].dropna(1)
    return data
# Funcion -decile- segmenta la "data" entregada en deciles.
def decile(data):
    averageRet = np.mean(data)
    decile = pd.qcut(averageRet,10)
    return decile

#def schedule(data,decile):
#    averageRet = np.mean(data)
#    portfolio_return_average = averageRet.groupby(decile).mean().tolist()
    #resume.append(portfolio_return_average)
#    return portfolio_return_average

# Funcion -newtable- crea un data frame a partir del input.
def newtable(ave_ret):
    NT = pd.DataFrame(ave_ret)
    return NT

In [6]:
resume_test = []
resume_sub = []
resume_for = []
pct = []
pct_totales =[]
z = 0
k = 1
for i in range(0,48):
    # Estrategía de Carhart formación de carteras acorde a una ventana especifica (12 meses previos).
    # Primero obtenemos los fondos vigentes en la primera ventana con la función "boots".
    i = i*12
    primeros12 = boots(df, i, i+12)
    
    # Segundo clasificamos los fondos vigentes por deciles con la función "decile".
    f_deciles = decile(primeros12)
    # Trabajamos en df para facilitar el manejo de la información.
    test12 = newtable(f_deciles)
    
    # Tercero obtenemos los fondos vigentes en el mes subsecuente a la ventana anterior.
    # Ej(Ventana anterior terminaba el mes 12, se busca los fondos vigentes unicamente del mes 13).
    z = z + 12
    test_sub = boots(df,z,z+1)
    
    # Cuarto dado que tenemos los fondos vigentes durante 12 meses (test12) y los vigentes unicamente
    # en el mes 13 (test_sub). Se cruza la información para saber que fondos del test12 siguen vigentes en
    # test_sub junto con su retorno, de esta manera si un fondo expira no es un problema.
    testfinal = pd.concat([test12, test_sub.T], axis = 1).dropna()
    
    
    ############ Pregunta 1 (c) ##############
    modified = testfinal.reset_index()

    boolean_list = modified.groupby([0])[z].apply(list).apply(pd.Series)> market[z]
    fund_list = modified.groupby([0])['index'].apply(list).apply(pd.Series)
    fund_list_tot = fund_list.T.count().tolist()

    for j in range(0,10):
        pct.append(sum(boolean_list.iloc[j])/fund_list_tot[j])
    ###########################################
    

    # Quinto se agrupan los nuevos retornos en sus respectivos intervalos (criterio para formar el portfolio)
    # y se obtiene el promedio de cada portfolio EW.
    p_test_sub = testfinal.groupby(testfinal[0]).mean().reset_index()[z]#
    
    # Sexto evalua los fondos seleccionados para el mes de formación.
    test_for = boots(df,z-1,z)
    decile_for = decile(test_for)
    mean_for = np.mean(test_for)
    p_eva_for = mean_for.groupby(decile_for).mean().reset_index()[0]#
    
    
    # Finalmente se guardan los resultados de los Excesos de Retornos Mensuales
    resume_test.append(p_test_sub)
    resume_for.append(p_eva_for)
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

/anaconda3/lib/python3.6/site-pack

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

/anaconda3/lib/python3.6/site-pack

KeyError: 'Column not found: 576'

In [7]:
# Se ordena la información del for loop en data frames para trabajar facilmente.
df_resume_test = newtable(resume_test)
df_resume_for = newtable(resume_for)
p1_10_test = df_resume_test[9] - df_resume_test[0] #Label 9 Porfolio Ganador (1 High) - Label 0 Portfolio Perdedor (10 Low)
p1_10_for = df_resume_for[9] - df_resume_for[0] #Label 9 Porfolio Ganador (1 High) - Label 0 Portfolio Perdedor (10 Low)

p_spread = newtable([np.mean(p1_10_for), np.std(p1_10_for), np.mean(p1_10_test), np.std(p1_10_test)]).T

In [8]:
# TABLA PREGUNTA 1
pregunta1 = pd.DataFrame()
pregunta1['Retorno Mensual Subsecuente [%]'] = df_resume_test.mean()
pregunta1['Std Dev Subsecuente [%]'] = df_resume_test.std()

new_row = {'Retorno Mensual Subsecuente [%]':np.mean(p1_10_test), 'Std Dev Subsecuente [%]':np.std(p1_10_test)}
#append row to the dataframe
df_pregunta1 = pregunta1.append(new_row, ignore_index=True)
df_pregunta1.rename(index={ 0: '10 (Low)', 1: '9', 2:'8',3:'7',4:'6',5:'5', 6:'4', 7:'3', 8:'2', 9:'1 (High)', 10: '1-10 spread'}, inplace=True)

df_pregunta1

,Retorno Mensual Subsecuente [%],Std Dev Subsecuente [%]
10 (Low),1.042230,5.109541
9,0.942889,4.763487
8,0.935478,4.803577
7,0.841113,4.697618
6,0.877347,4.777783
5,0.989169,4.850431
4,0.987979,4.837280
3,1.108841,4.950659
2,1.490309,5.474118
1 (High),1.744956,5.669431


In [9]:
# TABLA PREGUNTA 2
schedule = pd.DataFrame()
schedule['Retorno Mensual Formacion [%]'] = df_resume_for.mean()
schedule['Std Dev Formacion [%]'] = df_resume_for.std()
schedule['Retorno Mensual Subsecuente [%]'] = df_resume_test.mean()
schedule['Std Dev Subsecuente [%]'] = df_resume_test.std()
#schedule.rename(index={ 0: '10 (Low)', 1: '9', 2:'8',3:'7',4:'6',5:'5', 6:'4', 7:'3', 8:'2', 9:'1 (High)', 10: '1-10 spread'}, inplace=True)

new_row = {'Retorno Mensual Formacion [%]':np.mean(p1_10_for), 'Std Dev Formacion [%]':np.std(p1_10_for), 'Retorno Mensual Subsecuente [%]':np.mean(p1_10_test), 'Std Dev Subsecuente [%]':np.std(p1_10_test)}
#append row to the dataframe
df_final = schedule.append(new_row, ignore_index=True)
df_final.rename(index={ 0: '10 (Low)', 1: '9', 2:'8',3:'7',4:'6',5:'5', 6:'4', 7:'3', 8:'2', 9:'1 (High)', 10: '1-10 spread'}, inplace=True)

df_final

,Retorno Mensual Formacion [%],Std Dev Formacion [%],Retorno Mensual Subsecuente [%],Std Dev Subsecuente [%]
10 (Low),-4.314367,6.053735,1.042230,5.109541
9,-0.697498,3.040371,0.942889,4.763487
8,0.106206,2.997887,0.935478,4.803577
7,0.679819,3.066953,0.841113,4.697618
6,1.167720,3.161016,0.877347,4.777783
5,1.661384,3.281148,0.989169,4.850431
4,2.224014,3.463355,0.987979,4.837280
3,2.948598,3.736586,1.108841,4.950659
2,4.177566,4.288501,1.490309,5.474118
1 (High),9.644562,7.738177,1.744956,5.669431


In [12]:
# TABLA PREGUNTA 3
pctP = pd.DataFrame()
z = 11
for i in range (0,42):
    year = 1963 + i
    pctP['Formation %s winners'% year] = pct[i*z:10+(z*i)]

pctP.rename(index={ 0: '10 (Low)', 1: '9', 2:'8',3:'7',4:'6',5:'5', 6:'4', 7:'3', 8:'2', 9:'1 (High)'}, inplace=True)

# Analisis de en promedio cuantos fondos en los portfolios les ganan al mercado ei: W10 corresponde al P10.
winners = pd.DataFrame()
win10 = np.mean(pctP.iloc[0])
win9 = np.mean(pctP.iloc[1])
win8 = np.mean(pctP.iloc[2])
win7 = np.mean(pctP.iloc[3])
win6 = np.mean(pctP.iloc[4])
win5 = np.mean(pctP.iloc[5])
win4 = np.mean(pctP.iloc[6])
win3 = np.mean(pctP.iloc[7])
win2 = np.mean(pctP.iloc[8])
win1 = np.mean(pctP.iloc[9])
pctP

,Formation 1963 winners,Formation 1964 winners,Formation 1965 winners,Formation 1966 winners,Formation 1967 winners,Formation 1968 winners,Formation 1969 winners,Formation 1970 winners,Formation 1971 winners,Formation 1972 winners,...,Formation 1995 winners,Formation 1996 winners,Formation 1997 winners,Formation 1998 winners,Formation 1999 winners,Formation 2000 winners,Formation 2001 winners,Formation 2002 winners,Formation 2003 winners,Formation 2004 winners
10 (Low),0.500000,0.166667,0.692308,1.000000,0.428571,0.357143,0.666667,0.636364,0.480000,1.000000,...,0.314465,0.138122,0.482927,0.252336,0.558952,0.358974,0.769565,0.532189,0.646288,0.662100
9,0.500000,0.333333,0.692308,0.785714,0.285714,0.500000,0.631579,0.727273,0.576923,0.034483,...,0.234177,0.204420,0.357843,0.260465,0.591304,0.434043,0.814655,0.391304,0.588745,0.729358
8,0.545455,0.230769,0.461538,0.928571,0.357143,0.307692,0.631579,0.695652,0.413793,0.233333,...,0.377358,0.226519,0.524272,0.347222,0.748918,0.514894,0.383621,0.318777,0.597403,0.637615
7,0.166667,0.250000,0.461538,1.000000,0.357143,0.230769,0.631579,0.692308,0.357143,0.233333,...,0.345912,0.364641,0.687805,0.314815,0.848485,0.090909,0.493562,0.278261,0.552174,0.625571
6,0.400000,0.250000,0.384615,0.928571,0.785714,0.466667,0.476190,0.791667,0.448276,0.200000,...,0.405063,0.591160,0.736585,0.314815,0.632479,0.138528,0.599138,0.391304,0.489177,0.493151
5,0.333333,0.307692,0.615385,1.000000,0.642857,0.210526,0.409091,0.680000,0.642857,0.233333,...,0.396226,0.607735,0.839806,0.259740,0.635193,0.194805,0.663793,0.391304,0.450216,0.264840
4,0.363636,0.416667,0.461538,0.923077,0.866667,0.263158,0.590909,0.560000,0.551724,0.103448,...,0.540881,0.912088,0.558140,0.239130,0.519149,0.168831,0.582609,0.513043,0.359307,0.196347
3,0.416667,0.166667,0.769231,0.769231,0.571429,0.444444,0.500000,0.640000,0.535714,0.300000,...,0.421384,0.397059,0.425926,0.313043,0.444444,0.281385,0.545852,0.630435,0.389610,0.068493
2,0.545455,0.615385,0.214286,0.571429,0.500000,0.444444,0.636364,0.640000,0.750000,0.166667,...,0.076923,0.507317,0.231481,0.423581,0.269231,0.400000,0.583691,0.694323,0.346320,0.090909
1 (High),0.500000,0.857143,0.642857,0.500000,0.357143,0.473684,0.523810,0.520000,0.892857,0.241379,...,0.104972,0.497561,0.306977,0.588745,0.382979,0.556034,0.586207,0.826087,0.327586,0.712919


In [10]:
# REGRESION CAPM
def capmRegression (df_resume_test, market):
    
    p10 = df_resume_test[0].tolist()
    p9 = df_resume_test[1].tolist()
    p8 = df_resume_test[2].tolist()
    p7 = df_resume_test[3].tolist()
    p6 = df_resume_test[4].tolist()
    p5 = df_resume_test[5].tolist()
    p4 = df_resume_test[6].tolist()
    p3 = df_resume_test[7].tolist()
    p2 = df_resume_test[8].tolist()
    p1 = df_resume_test[9].tolist()
    p1_10 = (df_resume_test[9] - df_resume_test[0]).tolist()
    
    market_sub = []
    for i in range(1,48):
        vector = market[i*12]
        market_sub.append(vector)

    capm = pd.DataFrame()
    capm['Rm'] = market_sub
    capm['P10'] = p10
    capm['P9'] = p9
    capm['P8'] = p8
    capm['P7'] = p7
    capm['P6'] = p6
    capm['P5'] = p5
    capm['P4'] = p4
    capm['P3'] = p3
    capm['P2'] = p2
    capm['P1'] = p1
    capm['1-10 Spread'] = p1_10
    
    # CAPM Regression P10 vs Market
    monthly_p10 = pd.concat([capm['P10'], capm['Rm']], axis=1)
    y10 = monthly_p10['P10']
    X = monthly_p10['Rm']
    X1 = sm.add_constant(X)
    model = sm.OLS(y10, X1)
    results = model.fit()
    slope10, intercept10, r_value, p_value, std_err = stats.linregress(X, y10)
    adj_r2_p10 = results.rsquared_adj

    betas = []
    alphas = []
    adj_R = []
    betas.append(slope10)
    alphas.append(intercept10)
    adj_R.append(adj_r2_p10)
    
    # CAPM Regression P9 vs Market
    monthly_p9 = pd.concat([capm['P9'], capm['Rm']], axis=1)
    y9 = monthly_p9['P9']
    model9 = sm.OLS(y9, X1)
    results9 = model9.fit()
    slope9, intercept9, r_value, p_value, std_err = stats.linregress(X, y9)
    adj_r2_p9 = results.rsquared_adj

    betas.append(slope9)
    alphas.append(intercept9)
    adj_R.append(adj_r2_p9)
    
    # CAPM Regression P8 vs Market
    monthly_p8 = pd.concat([capm['P8'], capm['Rm']], axis=1)
    y8 = monthly_p8['P8']
    model = sm.OLS(y8, X1)
    results = model.fit()
    slope8, intercept8, r_value, p_value, std_err = stats.linregress(X, y8)
    adj_r2_p8 = results.rsquared_adj

    betas.append(slope8)
    alphas.append(intercept8)
    adj_R.append(adj_r2_p8)
    
    # CAPM Regression P7 vs Market
    monthly_p7 = pd.concat([capm['P7'], capm['Rm']], axis=1)
    y7 = monthly_p7['P7']
    model = sm.OLS(y7, X1)
    results = model.fit()
    slope7, intercept7, r_value, p_value, std_err = stats.linregress(X, y7)
    adj_r2_p7 = results.rsquared_adj

    betas.append(slope7)
    alphas.append(intercept7)
    adj_R.append(adj_r2_p7)
    
    # CAPM Regression P6 vs Market
    monthly_p6 = pd.concat([capm['P6'], capm['Rm']], axis=1)
    y6 = monthly_p6['P6']
    model = sm.OLS(y6, X1)
    results = model.fit()
    slope6, intercept6, r_value, p_value, std_err = stats.linregress(X, y6)
    adj_r2_p6 = results.rsquared_adj
    
    betas.append(slope6)
    alphas.append(intercept6)
    adj_R.append(adj_r2_p6)
    
    # CAPM Regression P5 vs Market
    monthly_p5 = pd.concat([capm['P5'], capm['Rm']], axis=1)
    y5 = monthly_p5['P5']
    model = sm.OLS(y5, X1)
    results = model.fit()
    slope5, intercept5, r_value, p_value, std_err = stats.linregress(X, y5)
    adj_r2_p5 = results.rsquared_adj

    betas.append(slope5)
    alphas.append(intercept5)
    adj_R.append(adj_r2_p5)
    
    # CAPM Regression P4 vs Market
    monthly_p4 = pd.concat([capm['P4'], capm['Rm']], axis=1)
    y4 = monthly_p4['P4']
    model = sm.OLS(y4, X1)
    results = model.fit()
    slope4, intercept4, r_value, p_value, std_err = stats.linregress(X, y4)
    adj_r2_p4 = results.rsquared_adj

    betas.append(slope4)
    alphas.append(intercept4)
    adj_R.append(adj_r2_p4)
    
    # CAPM Regression P3 vs Market
    monthly_p3 = pd.concat([capm['P3'], capm['Rm']], axis=1)
    y3 = monthly_p3['P3']
    model = sm.OLS(y3, X1)
    results = model.fit()
    slope3, intercept3, r_value, p_value, std_err = stats.linregress(X, y3)
    adj_r2_p3 = results.rsquared_adj

    betas.append(slope3)
    alphas.append(intercept3)
    adj_R.append(adj_r2_p3)
    
    # CAPM Regression P2 vs Market
    monthly_p2 = pd.concat([capm['P2'], capm['Rm']], axis=1)
    y2 = monthly_p2['P2']
    model = sm.OLS(y2, X1)
    results = model.fit()
    slope2, intercept2, r_value, p_value, std_err = stats.linregress(X, y2)
    adj_r2_p2 = results.rsquared_adj

    betas.append(slope2)
    alphas.append(intercept2)
    adj_R.append(adj_r2_p2)
    
    # CAPM Regression P1 vs Market
    monthly_p1 = pd.concat([capm['P1'], capm['Rm']], axis=1)
    y1 = monthly_p1['P1']
    model = sm.OLS(y1, X1)
    results = model.fit()
    slope1, intercept1, r_value, p_value, std_err = stats.linregress(X, y1)
    adj_r2_p1 = results.rsquared_adj
    
    betas.append(slope1)
    alphas.append(intercept1)
    adj_R.append(adj_r2_p1)
    
    # CAPM Regression Spread 1-10 vs Market
    monthly_p1_10 = pd.concat([capm['1-10 Spread'], capm['Rm']], axis=1)
    y1_10 = monthly_p1_10['1-10 Spread']
    model = sm.OLS(y1_10, X1)
    results = model.fit()

    slope1_10, intercept1_10, r_value, p_value, std_err = stats.linregress(X, y1_10)
    adj_r2_p1_10 = results.rsquared_adj

    betas.append(slope1_10)
    alphas.append(intercept1_10)
    adj_R.append(adj_r2_p1_10)
    
    return (betas, alphas, adj_R)

In [11]:
# OUTPUT REGRESION CAPM
regresores = capmRegression (df_resume_test, market)
betas = regresores[0]
alphas = regresores[1]
adj_R = regresores[2]
df_final['Alpha'] = alphas
df_final['Beta'] = betas
df_final['Adj R-sq'] = adj_R
df_final

,Retorno Mensual Formacion [%],Std Dev Formacion [%],Retorno Mensual Subsecuente [%],Std Dev Subsecuente [%],Alpha,Beta,Adj R-sq
10 (Low),-4.314367,6.053735,1.042230,5.109541,-0.076009,0.904288,0.825390
9,-0.697498,3.040371,0.942889,4.763487,-0.154595,0.887505,0.825390
8,0.106206,2.997887,0.935478,4.803577,-0.191981,0.911744,0.952686
7,0.679819,3.066953,0.841113,4.697618,-0.261276,0.891471,0.952333
6,1.167720,3.161016,0.877347,4.777783,-0.254346,0.915168,0.970656
5,1.661384,3.281148,0.989169,4.850431,-0.151454,0.922389,0.956400
4,2.224014,3.463355,0.987979,4.837280,-0.139239,0.911549,0.938737
3,2.948598,3.736586,1.108841,4.950659,-0.024098,0.916176,0.904563
2,4.177566,4.288501,1.490309,5.474118,0.257335,0.997071,0.875561
1 (High),9.644562,7.738177,1.744956,5.669431,0.564472,0.954624,0.745021


In [12]:
# REGRESION MODELO DE 4 FACTORES
def carhartRegression (df_resume_test, market, smb, hml, umd):
    
    p10 = df_resume_test[0].tolist()
    p9 = df_resume_test[1].tolist()
    p8 = df_resume_test[2].tolist()
    p7 = df_resume_test[3].tolist()
    p6 = df_resume_test[4].tolist()
    p5 = df_resume_test[5].tolist()
    p4 = df_resume_test[6].tolist()
    p3 = df_resume_test[7].tolist()
    p2 = df_resume_test[8].tolist()
    p1 = df_resume_test[9].tolist()
    p1_10 = (df_resume_test[9] - df_resume_test[0]).tolist()

    df = pd.DataFrame()
    df['MKT'] = market.tolist()
    df['SMB'] = smb[0].tolist()
    df['HML'] = hml[0].tolist()
    df['UMD'] = umd[0].tolist()

    carhart_sub = []
    for i in range(1,48):
        vector = df.iloc[i*12]
        carhart_sub.append(vector)

    factors = newtable(carhart_sub).reset_index(drop=True)

    df_carhart = pd.DataFrame()
    df_carhart['P10'] = p10
    df_carhart['P9'] = p9
    df_carhart['P8'] = p8
    df_carhart['P7'] = p7
    df_carhart['P6'] = p6
    df_carhart['P5'] = p5
    df_carhart['P4'] = p4
    df_carhart['P3'] = p3
    df_carhart['P2'] = p2
    df_carhart['P1'] = p1
    df_carhart['1-10 Spread'] = p1_10
    #df_final = pd.merge(df_carhart,factors,left_index=True,right_index=True)
    
    
    
    
    # 4 Factors Regression P1 vs Market
    y1 = df_carhart['P1']
    X = factors
    X1 = sm.add_constant(X)
    model = sm.OLS(y1, X1)
    results = model.fit()
    coef1 = results.params.tolist()
    coef1.append(results.rsquared_adj)
    
    # 4 Factors Regression P2 vs Market
    y2 = df_carhart['P2']
    model = sm.OLS(y2, X1)
    results = model.fit()
    coef2 = results.params.tolist()
    coef2.append(results.rsquared_adj)
    
    # 4 Factors Regression P3 vs Market
    y3 = df_carhart['P3']
    model = sm.OLS(y3, X1)
    results = model.fit()
    coef3 = results.params.tolist()
    coef3.append(results.rsquared_adj)
    
    # 4 Factors Regression P4 vs Market
    y4 = df_carhart['P4']
    model = sm.OLS(y4, X1)
    results = model.fit()
    coef4 = results.params.tolist()
    coef4.append(results.rsquared_adj)
    
    # 4 Factors Regression P5 vs Market
    y5 = df_carhart['P5']
    model = sm.OLS(y5, X1)
    results = model.fit()
    coef5 = results.params.tolist()
    coef5.append(results.rsquared_adj)
    
    # 4 Factors Regression P6 vs Market
    y6 = df_carhart['P6']
    model = sm.OLS(y6, X1)
    results = model.fit()
    coef6 = results.params.tolist()
    coef6.append(results.rsquared_adj)
    
    # 4 Factors Regression P7 vs Market
    y7 = df_carhart['P7']
    model = sm.OLS(y7, X1)
    results = model.fit()
    coef7 = results.params.tolist()
    coef7.append(results.rsquared_adj)
    
    # 4 Factors Regression P8 vs Market
    y8 = df_carhart['P8']
    model = sm.OLS(y8, X1)
    results = model.fit()
    coef8 = results.params.tolist()
    coef8.append(results.rsquared_adj)
    
    # 4 Factors Regression P9 vs Market
    y9 = df_carhart['P9']
    model = sm.OLS(y9, X1)
    results = model.fit()
    coef9 = results.params.tolist()
    coef9.append(results.rsquared_adj)
    
    # 4 Factors Regression P10 vs Market
    y10 = df_carhart['P10']
    model = sm.OLS(y10, X1)
    results = model.fit()
    coef10 = results.params.tolist()
    coef10.append(results.rsquared_adj)
    
    # 4 Factors Regression 1-10 Spread vs Market
    y1_10 = df_carhart['1-10 Spread']
    model = sm.OLS(y1_10, X1)
    results = model.fit()
    coef1_10 = results.params.tolist()
    coef1_10.append(results.rsquared_adj)
    
    
    output = pd.DataFrame()
    outputfinal = pd.concat([output, newtable(coef10).T, newtable(coef9).T, newtable(coef8).T, newtable(coef7).T, newtable(coef6).T, newtable(coef5).T, newtable(coef4).T, newtable(coef3).T, newtable(coef2).T, newtable(coef1).T, newtable(coef1_10).T], ignore_index=True)
    
    return outputfinal

In [13]:
# OUTPUT REGRESION MODELO DE 4 FACTORES
factor_model = carhartRegression (df_resume_test, market, smb, hml, umd)

df_final['Alpha2'] = factor_model[0].tolist()
df_final['MKT'] = factor_model[1].tolist()
df_final['SMB'] = factor_model[2].tolist()
df_final['HML'] = factor_model[3].tolist()
df_final['UMD'] = factor_model[4].tolist()
df_final['Adj R-Sq'] = factor_model[5].tolist()
df_final.drop(['Retorno Mensual Formacion [%]', 'Std Dev Formacion [%]'], axis=1)

,Retorno Mensual Subsecuente [%],Std Dev Subsecuente [%],Alpha,Beta,Adj R-sq,Alpha2,MKT,SMB,HML,UMD,Adj R-Sq
10 (Low),1.042230,5.109541,-0.076009,0.904288,0.825390,-0.526091,0.851723,0.164826,-0.031915,-0.161001,0.877242
9,0.942889,4.763487,-0.154595,0.887505,0.825390,-0.372610,0.857605,0.111579,-0.042181,-0.062321,0.929109
8,0.935478,4.803577,-0.191981,0.911744,0.952686,-0.398391,0.888065,0.108276,-0.016008,-0.022124,0.957704
7,0.841113,4.697618,-0.261276,0.891471,0.952333,-0.335987,0.878299,0.084427,-0.034862,0.026591,0.951889
6,0.877347,4.777783,-0.254346,0.915168,0.970656,-0.307892,0.901727,0.107843,-0.051140,0.061058,0.974857
5,0.989169,4.850431,-0.151454,0.922389,0.956400,-0.153937,0.913971,0.100535,-0.053695,0.089595,0.963115
4,0.987979,4.837280,-0.139239,0.911549,0.938737,-0.215992,0.897351,0.183824,-0.050019,0.167139,0.968077
3,1.108841,4.950659,-0.024098,0.916176,0.904563,-0.097999,0.890873,0.266469,-0.118894,0.211349,0.951833
2,1.490309,5.474118,0.257335,0.997071,0.875561,-0.071498,0.957656,0.357214,-0.054763,0.227734,0.926990
1 (High),1.744956,5.669431,0.564472,0.954624,0.745021,0.266031,0.889309,0.599550,-0.234647,0.406382,0.892641
